# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Part 2: NYC 311 calls [22 points]

This notebook derives from a [demo by the makers of plot.ly](https://plot.ly/ipython-notebooks/big-data-analytics-with-pandas-and-sqlite/). We've adapted it to use [Bokeh](http://bokeh.pydata.org/en/latest/).

You will start with a large database of complaints filed by residents of New York City since 2010 via 311 calls. The full dataset is available at the [NYC open data portal](https://nycopendata.socrata.com/data). At about 6 GB and 10 million complaints, you can infer that a) you might not want to read it all into memory at once, and b) NYC residents are really whiny. (Maybe only conclusion "a" is valid.) The notebook then combines the use of `sqlite`, `pandas`, and `bokeh`.

## Setup

Next, as a quick test let's make a simple plot.

In [ ]:
from IPython.display import display
import pandas as pd

The following module defines a couple functions from Lab 4 that we'll need again in this lab, in particular, `tibbles_are_equivalent()`.

In [ ]:
from cse6040utils import *

In [ ]:
# Build a Pandas data frame
names = ['Bob','Jessica','Mary','John','Mel']
births = [968, 155, 77, 578, 973]
name_birth_pairs = list (zip (names, births))
baby_names = pd.DataFrame(data=name_birth_pairs, columns=['Names', 'Births'])
display (baby_names)

In [ ]:
from bokeh.charts import Bar
from bokeh.io import show, output_notebook
output_notebook ()

In [ ]:
show (Bar (baby_names, values='Births', label='Names', color='Names', width=300, height=300, legend=None))

**Download a sample dataset.** Next, grab a copy of today's dataset, which is a small (~ 20%) subset of the full dataset:

* [SQLite DB, ~ 257 MiB] http://cse6040.gatech.edu/datasets/NYC-311-2M.db

Let's connect to this database.

In [ ]:
# SQLite database filename
DB_FILENAME = 'NYC-311-2M.db'

# Connect
import sqlite3 as db
disk_engine = db.connect (DB_FILENAME)

**Preview the data.** This sample database has just a single table, named `data`. Let's query it and see how long it takes to read. To carry out the query, we will use the SQL reader built into `pandas`.

In [ ]:
import time

In [ ]:
print ("Reading ...")
start_time = time.time ()

# Perform SQL query through the disk_engine connection.
# The return value is a pandas data frame.
df = pd.read_sql_query ('select * from data', disk_engine)

elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

# Dump the first few rows
df.head()

## More SQL stuff

**Partial queries: `LIMIT` clause.** The preceding command was overkill for what we wanted, which was just to preview the table. Instead, we could have used the `LIMIT` option to ask for just a few results.

In [ ]:
query = '''
  select *
    from data
    limit 5
'''
start_time = time.time ()
df = pd.read_sql_query (query, disk_engine)
elapsed_time = time.time () - start_time
print ("==> LIMIT version took %g seconds." % elapsed_time)

df

Let's use the `LIMIT` feature to create a generic function that "peeks" at the first few entries of any table.

In [ ]:
def peek_table (db, name, num=5):
    """
    Given a database connection (`db`), prints both the number of
    records in the table as well as its first few entries.
    """
    count = '''select count (*) FROM {table}'''.format (table=name)
    peek = '''select * from {table} limit {limit}'''.format (table=name, limit=num)
    
    print ("Total number of records:")
    display (pd.read_sql_query (count, db))
    
    print ("First {} entries:".format (num))
    display (pd.read_sql_query (peek, db))
    
peek_table (disk_engine, 'data')

**Set membership: `IN` operator.** Another common idiom is to ask for rows whose attributes fall within a set, for which you can use the `IN` operator.

In [ ]:
query = '''
  select ComplaintType, Descriptor, Agency
    from data
    where Agency in ("NYPD", "DOB")
    limit 10
'''

df = pd.read_sql_query (query, disk_engine)
df.head ()

**Finding unique values: `DISTINCT` qualifier.** Yet another common idiom is to ask for the unique values of some attribute, for which you can use the `DISTINCT` qualifier.

In [ ]:
query = 'select distinct City FROM data'
df = pd.read_sql_query (query, disk_engine)

print ("Found %d unique cities. The first few are:" % len (df))
df.head ()

**Renaming columns: `AS` operator.** Sometimes you might want to rename a result column. For instance, the following query counts the number of complaints by "Agency," using the `COUNT(*)` function and `GROUP BY` clause, which we discussed in an earlier lab. If you wish to refer to the counts column of the resulting data frame, you can give it a more "friendly" name using the `AS` operator.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
'''
df = pd.read_sql_query (query, disk_engine)
df.head ()

**Ordering results: `ORDER` clause.** You can also order the results. For instance, suppose we want to execute the previous query by number of complaints.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
    order by NumComplaints
'''
df = pd.read_sql_query (query, disk_engine)
df.tail ()

Note that the above example prints the bottom (tail) of the data frame. You could have also asked for the query results in reverse (descending) order, by prefixing the `ORDER BY` attribute with a `-` (minus) symbol.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
    order by -NumComplaints
'''
df = pd.read_sql_query (query, disk_engine)
df.head ()

And of course we can plot all of this data!

In [ ]:
# Disable auto-sorting x-axis:
#   https://github.com/bokeh/bokeh/issues/2924
# + http://bokeh.pydata.org/en/latest/docs/gallery/stacked_bar_chart.html
from bokeh.charts.attributes import cat

show (Bar (df[:20],
           title='Top 20 agencies by number of complaints',
           values='NumComplaints',
           label=cat (columns=['Agency'], sort=False),
           width=800, height=400, legend=False))

**Exercise 1** (2 points). Create a string, `query`, that contains an SQL query that returns the number of complaints by type. The columns should be named `type` and `freq`, and the results should be sorted in descending order by `freq`.

> What is the most common type of complaint? What, if anything, does it tell you about NYC?

In [ ]:
# Define a string, `query`, containing your query:
# YOUR CODE HERE
raise NotImplementedError()

# Runs your `query`:
df_complaint_freq = pd.read_sql_query (query, disk_engine)
df_complaint_freq.head ()

In [ ]:
print ("Top 10 complaints:")
display (df_complaint_freq.head (10))

soln = ['HEAT/HOT WATER', 'Street Condition', 'Street Light Condition', 'Blocked Driveway', 'Illegal Parking', 'UNSANITARY CONDITION', 'PAINT/PLASTER', 'Water System', 'PLUMBING', 'Noise', 'Noise - Street/Sidewalk', 'Traffic Signal Condition', 'Noise - Commercial', 'DOOR/WINDOW', 'WATER LEAK', 'Dirty Conditions', 'Sewer', 'Sanitation Condition', 'DOF Literature Request', 'ELECTRIC', 'Rodent', 'FLOORING/STAIRS', 'General Construction/Plumbing', 'Building/Use', 'Broken Muni Meter', 'GENERAL', 'Missed Collection (All Materials)', 'Benefit Card Replacement', 'Derelict Vehicle', 'Noise - Vehicle', 'Damaged Tree', 'Consumer Complaint', 'Derelict Vehicles', 'Taxi Complaint', 'Overgrown Tree/Branches', 'Graffiti', 'Snow', 'Opinion for the Mayor', 'APPLIANCE', 'Maintenance or Facility', 'Animal Abuse', 'Dead Tree', 'HPD Literature Request', 'Root/Sewer/Sidewalk Condition', 'SAFETY', 'Elevator', 'Food Establishment', 'SCRIE', 'Air Quality', 'Agency Issues', 'Construction', 'Highway Condition', 'Other Enforcement', 'Water Conservation', 'Sidewalk Condition', 'Indoor Air Quality', 'Street Sign - Damaged', 'Traffic', 'Plumbing', 'Fire Safety Director - F58', 'Homeless Person Assistance', 'Homeless Encampment', 'Special Enforcement', 'Street Sign - Missing', 'Noise - Park', 'Vending', 'For Hire Vehicle Complaint', 'Food Poisoning', 'Special Projects Inspection Team (SPIT)', 'Hazardous Materials', 'Electrical', 'DOT Literature Request', 'Litter Basket / Request', 'Taxi Report', 'Illegal Tree Damage', 'DOF Property - Reduction Issue', 'Unsanitary Animal Pvt Property', 'Asbestos', 'Lead', 'Vacant Lot', 'DCA / DOH New License Application Request', 'Street Sign - Dangling', 'Smoking', 'Violation of Park Rules', 'OUTSIDE BUILDING', 'Animal in a Park', 'Noise - Helicopter', 'School Maintenance', 'DPR Internal', 'Boilers', 'Industrial Waste', 'Sweeping/Missed', 'Overflowing Litter Baskets', 'Non-Residential Heat', 'Curb Condition', 'Drinking', 'Standing Water', 'Indoor Sewage', 'Water Quality', 'EAP Inspection - F59', 'Derelict Bicycle', 'Noise - House of Worship', 'DCA Literature Request', 'Recycling Enforcement', 'ELEVATOR', 'DOF Parking - Tax Exemption', 'Broken Parking Meter', 'Request for Information', 'Taxi Compliment', 'Unleashed Dog', 'Urinating in Public', 'Unsanitary Pigeon Condition', 'Investigations and Discipline (IAD)', 'Bridge Condition', 'Ferry Inquiry', 'Bike/Roller/Skate Chronic', 'Public Payphone Complaint', 'Vector', 'BEST/Site Safety', 'Sweeping/Inadequate', 'Disorderly Youth', 'Found Property', 'Mold', 'Senior Center Complaint', 'Fire Alarm - Reinspection', 'For Hire Vehicle Report', 'City Vehicle Placard Complaint', 'Cranes and Derricks', 'Ferry Complaint', 'Illegal Animal Kept as Pet', 'Posting Advertisement', 'Harboring Bees/Wasps', 'Panhandling', 'Scaffold Safety', 'OEM Literature Request', 'Plant', 'Bus Stop Shelter Placement', 'Collection Truck Noise', 'Beach/Pool/Sauna Complaint', 'Complaint', 'Compliment', 'Illegal Fireworks', 'Fire Alarm - Modification', 'DEP Literature Request', 'Drinking Water', 'Fire Alarm - New System', 'Poison Ivy', 'Bike Rack Condition', 'Emergency Response Team (ERT)', 'Municipal Parking Facility', 'Tattooing', 'Unsanitary Animal Facility', 'Animal Facility - No Permit', 'Miscellaneous Categories', 'Misc. Comments', 'Literature Request', 'Special Natural Area District (SNAD)', 'Highway Sign - Damaged', 'Public Toilet', 'Adopt-A-Basket', 'Ferry Permit', 'Invitation', 'Window Guard', 'Parking Card', 'Illegal Animal Sold', 'Stalled Sites', 'Open Flame Permit', 'Overflowing Recycling Baskets', 'Highway Sign - Missing', 'Public Assembly', 'DPR Literature Request', 'Fire Alarm - Addition', 'Lifeguard', 'Transportation Provider Complaint', 'DFTA Literature Request', 'Bottled Water', 'Highway Sign - Dangling', 'DHS Income Savings Requirement', 'Legal Services Provider Complaint', 'Foam Ban Enforcement', 'Tunnel Condition', 'Calorie Labeling', 'Fire Alarm - Replacement', 'X-Ray Machine/Equipment', 'Sprinkler - Mechanical', 'Hazmat Storage/Use', 'Tanning', 'Radioactive Material', 'Rangehood', 'SRDE', 'Squeegee', 'Building Condition', 'SG-98', 'Standpipe - Mechanical', 'AGENCY', 'Forensic Engineering', 'Public Assembly - Temporary', 'VACANT APARTMENT', 'Laboratory', 'SG-99']
assert all (soln == df_complaint_freq['type'])

print ("\n(Passed.)")

Let's also visualize the result, as a bar chart showing complaint types on the x-axis and the number of complaints on the y-axis.

In [ ]:
show (Bar (df_complaint_freq[:25],
           title='Top 25 complaints',
           values='freq',
           label=cat (columns=['type'], sort=False),
           width=800, height=400, legend=False))

**Simple substring matching: the `LIKE` operator.** Suppose we just want to look at the counts for all complaints that have the word `noise` in them. You can use the `LIKE` operator combined with the string wildcard, `%`, to look for case-insensitive substring matches.

In [ ]:
query = '''
  select ComplaintType as type, count(*) as freq
    from data
    where ComplaintType like '%noise%'
    group by type
    order by -freq
'''

df_noisy = pd.read_sql_query (query, disk_engine)
df_noisy

**Exercise 2** (2 points). Create a string variable, `query`, that contains an SQL query that will return the top 10 most "whiny" cities in descending order of number of complaints. It should return two columns, one named `name` holding the name of the city, and one named `freq` holding the number of complaints by that city.

In [ ]:
# Define a string, `query`, containing your query:
# YOUR CODE HERE
raise NotImplementedError()

# Runs your `query`:
df_whiny_cities = pd.read_sql_query (query, disk_engine)
df_whiny_cities

In [ ]:
assert df_whiny_cities['name'][0] == 'BROOKLYN'
assert df_whiny_cities['name'][1] == 'NEW YORK'
assert df_whiny_cities['name'][2] == 'BRONX'
assert df_whiny_cities['name'][3] is None
assert df_whiny_cities['name'][4] == 'STATEN ISLAND'

print ("\n(Passed partial test.)")

You should notice two bits of funny behavior, namely, that cities are treated in a _case-sensitive_ manner and that `None` appears as a city. (Presumably this setting occurs when a complaint is non-localized or the city is not otherwise specified.)

**Case-insensitive grouping: `COLLATE NOCASE`.** One way to carry out the preceding query in a case-insensitive way is to add a `COLLATE NOCASE` qualifier to the `GROUP BY` clause.

Let's filter out the 'None' cases as well, while we are at it.

In [ ]:
query = '''
  SELECT City as name, COUNT(*) AS freq
    FROM data
    WHERE name <> 'None'
    GROUP BY name COLLATE NOCASE
    ORDER BY -freq
    LIMIT 10
'''
df_whiny_cities2 = pd.read_sql_query (query, disk_engine)
df_whiny_cities2

Brooklyn is NYC's whiniest city. I knew it!

Lastly, for later use, let's save the names of just the top 7 cities.

In [ ]:
TOP_CITIES = list (df_whiny_cities2.head (7)['name'])
TOP_CITIES

**Exercise 3** (1 point). Implement a function that takes a list of strings, `str_list`, and returns a single string consisting of each value, `str_list[i]`, enclosed by double-quotes and separated by a comma-space delimiters. For example, if

```python
   assert str_list == ['a', 'b', 'c', 'd']
```

then

```python
   assert strs_to_args (str_list) == '"a", "b", "c", "d"'
```

In [ ]:
def strs_to_args (str_list):
    """Converts a list of strings to a comma-separated string argument list."""
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print ("Your solution, applied to TOP_CITIES:", strs_to_args (TOP_CITIES))

TOP_CITIES_as_args = strs_to_args (TOP_CITIES)
assert TOP_CITIES_as_args.lower () == \
       '"BROOKLYN", "NEW YORK", "BRONX", "STATEN ISLAND", "Jamaica", "Flushing", "ASTORIA"'.lower ()
    
print ("\n(Passed.)")

**Exercise 4** (3 points). Suppose we want to look at the number of complaints by type _and_ by city. Execute an SQL query to produce a tibble named `df_complaints_by_city` with the variables {`complaint_type`, `city_name`, `complaint_count`}.

In [ ]:
# Create a variable, `query`, containing your SQL query.
# YOUR CODE HERE
raise NotImplementedError()

# Previews the results of your query:
display (df_complaints_by_city.head (10))

In [ ]:
print ("Reading instructor's solution...")
df_complaints_by_city_soln = pd.read_csv ('df_complaints_by_city_soln.csv')

print ("Checking...")
assert tibbles_are_equivalent (df_complaints_by_city,
                               df_complaints_by_city_soln)

print ("\n(Passed.)")
del df_complaints_by_city_soln

Let's use Bokeh to visualize the results as a stacked bar chart.

In [ ]:
# Let's consider only the top 25 complaints (by total)
top_complaints = df_complaint_freq[:25]
display (top_complaints)

# Plot subset of data corresponding to the top complaints
df_plot = top_complaints.merge (df_complaints_by_city,
                                left_on=['type'],
                                right_on=['complaint_type'],
                                how='left')
display (df_plot.head ())

In [ ]:
# See also:
#   - http://bokeh.pydata.org/en/latest/docs/gallery/stacked_bar_chart.html
#   - http://stackoverflow.com/questions/33022985/data-tooltips-in-bokeh-dont-show-data-showing-instead

from bokeh.models import HoverTool

p = Bar (df_plot,
         values='complaint_count',
         label=cat (columns=['type'], sort=False),
         stack='city_name',
         color='city_name',
         width=800, height=400, legend='right_center',
         tools="hover,crosshair,pan,box_zoom,wheel_zoom,save,resize,reset,help")

hover_tool = p.select( dict (type=HoverTool))
hover_tool.tooltips = [('City', '@city_name'), ('Complaint', '@x'), ("#", "@height{int}")]

show (p)

**Exercise 5** (3 points). Make a variation of the above stacked bar chart that shows, for each complaint type (x-axis), the _percentage_ of complaints attributed to each city.

> Note: The normalized bars will not necessarily add up to 1. Why not?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Dates and times in SQL

Recall that the input data had a column with timestamps corresponding to when someone submitted a complaint. Let's quickly summarize some of the features in SQL and Python for reasoning about these timestamps.

The `CreatedDate` column is actually a specially formatted date and time stamp, where you can query against by comparing to strings of the form, `YYYY-MM-DD hh:mm:ss`.

For example, let's look for all complaints on September 15, 2015.

In [ ]:
query = '''
  select ComplaintType, CreatedDate, City
    from data
    where CreatedDate >= "2015-09-15 00:00:00.0"
      and CreatedDate < "2015-09-16 00:00:00.0"
    order by CreatedDate
'''
df = pd.read_sql_query (query, disk_engine)
df

This next example shows how to extract just the hour from the time stamp, using SQL's `strftime()`.

In [ ]:
query = '''
  select CreatedDate, strftime ('%H', CreatedDate) as Hour, ComplaintType
    from data
    limit 5
'''
df = pd.read_sql_query (query, disk_engine)
df

**Exercise 6** (3 points). Construct a tibble called `df_complaints_by_hour`, which contains the total number of complaints during a given hour of the day. That is, the variables should be {`hour`, `count`} where each observation is the total number of complaints (`count`) that occurred during a given `hour`.

> Interpret `hour` as follows: when `hour` is `02`, that corresponds to the open time interval [`02:00:00`, `03:00:00.0`).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

display (df_complaints_by_hour)

In [ ]:
print ("Reading instructor's solution...")
df_complaints_by_hour_soln = pd.read_csv ('df_complaints_by_hour_soln.csv')
display (df_complaints_by_hour_soln)

df_complaints_by_hour_norm = df_complaints_by_hour.copy ()
df_complaints_by_hour_norm['hour'] = \
    df_complaints_by_hour_norm['hour'].apply (int)
assert tibbles_are_equivalent (df_complaints_by_hour_norm,
                               df_complaints_by_hour_soln)
print ("\n(Passed.)")

In [ ]:
p = Bar (df_complaints_by_hour,
         values='count',
         label='hour',
         legend=None)
show (p)

An unusual aspect of these data are the excessively large number of reports associated with hour 0 (midnight up to but excluding 1 am). The reason is that there are some complaints that are dated but with no associated time, which then by default becomes `00:00:00.000`.

In [ ]:
query = '''
  select count (*)
    from data
    where strftime ('%H:%M:%f', CreatedDate) = '00:00:00.000'
'''
pd.read_sql_query (query, disk_engine)

**Exercise 7** (3 points). What is the most common hour for noise complaints? Compute a tibble called `df_noisy_by_hour` whose variables are {`hour`, `count`} and whose observations are the number of noise complaints that occurred during a given `hour`. Consider a "noise complaint" to be any complaint string containing the word `noise`. Be sure to filter out any dates _without_ an associated time, i.e., a timestamp of `00:00:00.000`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

display (df_noisy_by_hour)

In [ ]:
print ("Reading instructor's solution...")
df_noisy_by_hour_soln = pd.read_csv ('df_noisy_by_hour_soln.csv')
display (df_noisy_by_hour_soln)

df_noisy_by_hour_norm = df_noisy_by_hour.copy ()
df_noisy_by_hour_norm['hour'] = \
    df_noisy_by_hour_norm['hour'].apply (int)
assert tibbles_are_equivalent (df_noisy_by_hour_norm,
                               df_noisy_by_hour_soln)
print ("\n(Passed.)")

In [ ]:
p = Bar (df_noisy_by_hour,
         values='count',
         label='hour',
         legend=None)
show (p)

**Exercise 8** (5 points). Create a line chart (see [Line](http://bokeh.pydata.org/en/latest/docs/reference/charts.html#line)) to show the fraction of complaints (y-axis) associated with each hour of the day (x-axis), with each complaint type shown as a differently colored line. Show just the top 5 complaints (`top_complaints[:5]`). Remember to exclude complaints with a zero-timestamp (i.e., `00:00:00.000`).

In [ ]:
from bokeh.charts import Line

# YOUR CODE HERE
raise NotImplementedError()

### Learn more

- Find more open data sets on [Data.gov](https://data.gov) and [NYC Open Data](https://nycopendata.socrata.com)
- Learn how to setup [MySql with Pandas and Plotly](http://moderndata.plot.ly/graph-data-from-mysql-database-in-python/)
- Big data workflows with [HDF5 and Pandas](http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas)